In [122]:
import numpy as np
import pandas as pd

1. load the file
2. create universe dataframe
3. create buy sell signal
4. price all options

In [2]:
df_spx=pd.read_csv('SPX.csv')  
df_spx['date']=pd.to_datetime(df_spx['date'])
df_spx.index=df_spx['date']
df_spx=df_spx.sort_index()

In [123]:
df_bvols=pd.read_excel('bloomberg_spx_vols_values.xlsx', index_col=0) 
#df_bvols['Date']=pd.to_datetime(df_bvols['Date'])
#df_bvols.index=df_bvols['date']
#df_bvols=df_bvols.sort_index()
df_bvols=df_bvols['2005-05-18':]
df_bvols

,px,12m atm vol,24m atm vol,12m slope,24m slope
date,,,,,
2005-05-18,1185.56,14.7994,16.1761,-0.830764,-2.561531
2005-05-19,1191.08,14.4530,15.7034,-0.832010,-2.310149
2005-05-20,1189.28,14.3018,15.6604,-0.847508,-2.447997
2005-05-23,1193.86,14.0931,15.3672,-0.861660,-2.837509
2005-05-24,1194.07,14.0674,15.4045,-0.840980,-2.289923
...,...,...,...,...,...
2020-10-15,3483.34,22.2533,21.6859,-3.730241,-3.464660
2020-10-16,3483.81,22.5848,21.8534,-3.816402,-3.597786
2020-10-19,3426.92,23.4197,22.4914,-3.765921,-3.487846


In [124]:
pct_strikes=np.array([0.92,1,1,1.08])
maturity_days=720
maturity_years=2
price_field="px"
vol12_field="12m atm vol"
vol24_field="24m atm vol"
slope12_field="12m slope"
slope24_field="24m slope"
spot_field="px"
df_bvols[vol12_field]=df_bvols[vol12_field]/100
df_bvols[vol24_field]=df_bvols[vol24_field]/100
df_bvols[slope12_field]=df_bvols[slope12_field]/100
df_bvols[slope24_field]=df_bvols[slope24_field]/100

df_bvols.replace(0, np.nan, inplace=True)
df_bvols=df_bvols.fillna(method='ffill')
df_option_universe=pd.DataFrame(index=df_bvols.index)
strikes_universe=[]
creation_dates_universe=[]
maturity_dates_universe=[]
pc_universe=[]

for ind, row in df_bvols.iterrows():
    strikes=pct_strikes*row[price_field]
    pc=['p']*2+['c']*2
    creation_dates=[ind]*4
    maturity_dates=[ind+pd.Timedelta(days=maturity_years*365)]*4
    maturity_dates=pd.to_datetime(maturity_dates)
    strikes_universe.extend(strikes)
    creation_dates_universe.extend(creation_dates)
    maturity_dates_universe.extend(maturity_dates)
    pc_universe.extend(pc)

    #df_option_universe.loc[ind,'strikes']=strikes
    
    #if ind.day==1:
    #    df_spx.loc[ind,'pct_strike']=pct_strikes[np.random.randint(0,4)]
    #    df_spx.loc[ind,'maturity_days']=maturity_days
    #    maturity_days+=90


# create grids of strikes, creation dates, maturity dates, and pc]

strikes_universe=np.array(strikes_universe)
creation_dates_universe=np.array(creation_dates_universe)
maturity_dates_universe=np.array(maturity_dates_universe)
pc_universe=np.array(pc_universe)

#create 2d grids of strikes, creation dates, maturity dates, and pc where one dimension is the index of bvols and the other is the index of the strike
strikes_grid=np.tile(strikes_universe, (len(df_bvols),1))
creation_dates_grid=np.tile(creation_dates_universe, (len(df_bvols),1))
#need to figure out whiy the following needed to be done (convert back to datetime)
maturity_dates_grid=np.tile(maturity_dates_universe, (len(df_bvols),1))
pc_grid=np.tile(pc_universe, (len(df_bvols),1))
spot_grid=np.tile(df_bvols[spot_field].values, (len(strikes_universe),1)).T
vol12_grid=np.tile(df_bvols[vol12_field].values, (len(strikes_universe),1)).T
vol24_grid=np.tile(df_bvols[vol24_field].values, (len(strikes_universe),1)).T
slope12_grid=np.tile(df_bvols[slope12_field].values, (len(strikes_universe),1)).T
slope24_grid=np.tile(df_bvols[slope24_field].values, (len(strikes_universe),1)).T
date_grid=np.tile(df_bvols.index.to_numpy(), (len(strikes_universe),1)).T

In [125]:
myfunc_vec = np.vectorize(lambda x: x.to_datetime64())
texp_grid=(myfunc_vec(maturity_dates_grid) -date_grid)/np.timedelta64(1, 'D')/365


In [8]:
np.where(texp_grid<0, np.nan, texp_grid)

array([[ 2.        ,  2.        ,  2.        , ..., 17.43835616,
        17.43835616, 17.43835616],
       [ 1.99726027,  1.99726027,  1.99726027, ..., 17.43561644,
        17.43561644, 17.43561644],
       [ 1.99452055,  1.99452055,  1.99452055, ..., 17.43287671,
        17.43287671, 17.43287671],
       ...,
       [        nan,         nan,         nan, ...,  2.00547945,
         2.00547945,  2.00547945],
       [        nan,         nan,         nan, ...,  2.00273973,
         2.00273973,  2.00273973],
       [        nan,         nan,         nan, ...,  2.        ,
         2.        ,  2.        ]])

In [149]:
from scipy import interpolate
#interp_date_grid, interp_term_grid=np.meshgrid(df_bvols.index, [1,2],indexing='ij')
#create a table of atm vols and slopes for each date.
#rows are dates, columns are terms
#then we can go row by row and interpolate the vols and slopes for each instrument maturit, then use the interpolated vols and slopes to calculate the vol of each instrument
ivols_for_interp=df_bvols[[vol12_field,vol12_field, vol24_field,vol24_field]].values
slops_for_interp=df_bvols[[slope12_field,slope12_field, slope24_field,slope24_field]].values
#vol_interp=interpolate.RegularGridInterpolator((interp_date_grid, interp_term_grid), ivols_for_interp, method='linear')
#interp_date_grid.shape

# using giant negative value for initial interp point, need to fix, so that we dont have terms below zero
interp_terms=np.array([-50000,1,2,10000])
strike_vol_grid=np.zeros_like(strikes_grid)
strike_slope_grid=np.zeros_like(strikes_grid)

term_vol_grid=np.zeros_like(strikes_grid)
term_slope_grid=np.zeros_like(strikes_grid)

for i in range(len(df_bvols)):
    vol_interp=interpolate.interp1d(interp_terms, ivols_for_interp[i,:], kind='linear')
    slop_interp=interpolate.interp1d(interp_terms, slops_for_interp[i,:], kind='linear')
    term_vol_grid[i,:]=vol_interp(texp_grid[i,:])
    term_slope_grid[i,:]=slop_interp(texp_grid[i,:])

#compute spot vols from term vols and slopes 
#spot_vol_grid=term_vol_grid+term_slope_grid*np.log(strikes_grid/spot_grid)*10/np.sqrt(texp_grid)
spot_vol_grid=np.where(texp_grid<=0, np.nan, term_vol_grid+term_slope_grid*np.log(strikes_grid/spot_grid)*10/np.sqrt(texp_grid))
spot_vol_grid

/var/folders/c1/dv7w7b2x49j3spqsxnz6y_sw0000gn/T/ipykernel_8031/3305075419.py:27: RuntimeWarning: invalid value encountered in sqrt
  spot_vol_grid=np.where(texp_grid<=0, np.nan, term_vol_grid+term_slope_grid*np.log(strikes_grid/spot_grid)*10/np.sqrt(texp_grid))
/var/folders/c1/dv7w7b2x49j3spqsxnz6y_sw0000gn/T/ipykernel_8031/3305075419.py:27: RuntimeWarning: divide by zero encountered in divide
  spot_vol_grid=np.where(texp_grid<=0, np.nan, term_vol_grid+term_slope_grid*np.log(strikes_grid/spot_grid)*10/np.sqrt(texp_grid))


array([[0.17686371, 0.161761  , 0.161761  , ..., 0.09649694, 0.09649694,
        0.09177612],
       [0.17136375, 0.15775774, 0.15775774, ..., 0.09842716, 0.09842716,
        0.09416929],
       [0.171472  , 0.15707065, 0.15707065, ..., 0.09440651, 0.09440651,
        0.08989422],
       ...,
       [       nan,        nan,        nan, ..., 0.22429383, 0.22429383,
        0.20533903],
       [       nan,        nan,        nan, ..., 0.22423448, 0.22423448,
        0.20461059],
       [       nan,        nan,        nan, ..., 0.222671  , 0.222671  ,
        0.20375584]])

In [150]:
import py_vollib.black_scholes
import py_vollib_vectorized

In [175]:

price=py_vollib_vectorized.models.vectorized_black_scholes_merton(pc_grid.flatten(),spot_grid.flatten(),strikes_grid.flatten(),texp_grid.flatten(),0,spot_vol_grid.flatten(),0,return_as='array')
price=price.reshape(spot_grid.shape)
option_greeks = py_vollib_vectorized.get_all_greeks(pc_grid.flatten(), spot_grid, strikes_grid, texp_grid, 0, spot_vol_grid,0,model='black_scholes_merton', return_as='dict')


In [158]:
option_greeks={k: np.array(v).reshape(spot_grid.shape) for k, v in option_greeks.items()}


In [182]:

# do pl attribution
spot_lag_grid=np.roll(spot_grid,shift =1, axis=0)
spot_vol_lag_grid=np.roll(spot_vol_grid,shift =1, axis=0)
#texp_lag_grid=np.roll(texp_grid,shift =1, axis=0)
for grid in [spot_lag_grid,spot_vol_lag_grid]:
    grid[0,:]=np.nan

price_open=py_vollib_vectorized.models.vectorized_black_scholes_merton(pc_grid.flatten(),spot_lag_grid,strikes_grid,texp_grid,0,spot_vol_lag_grid.flatten(),0,return_as='array')
price_open=price_open.reshape(spot_grid.shape)
option_greeks_open=py_vollib_vectorized.get_all_greeks(pc_grid.flatten(), spot_lag_grid, strikes_grid, texp_grid, 0, spot_vol_lag_grid,0,model='black_scholes_merton', return_as='dict')
option_greeks_open={k: np.array(v).reshape(spot_grid.shape) for k, v in option_greeks_open.items()}
price_spot_move=py_vollib_vectorized.models.vectorized_black_scholes_merton(pc_grid.flatten(),spot_grid,strikes_grid,texp_grid,0,spot_vol_lag_grid,0,return_as='array')
price_spot_move=price_spot_move.reshape(spot_grid.shape)
spot_vol_lag_grid
#spot_grid.shape
#spot_grid
#grd

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [0.17686371, 0.161761  , 0.161761  , ..., 0.09649694, 0.09649694,
        0.09177612],
       [0.17136375, 0.15775774, 0.15775774, ..., 0.09842716, 0.09842716,
        0.09416929],
       ...,
       [       nan,        nan,        nan, ..., 0.22206995, 0.22206995,
        0.20255762],
       [       nan,        nan,        nan, ..., 0.22429383, 0.22429383,
        0.20533903],
       [       nan,        nan,        nan, ..., 0.22423448, 0.22423448,
        0.20461059]])

1) start with today (date), spot,vol (t-1) =>price_open
2) date, spot, today vol (t-1) => prices_spot_move

In [183]:
price_lag=np.roll(price,shift =1, axis=0)
price_lag[0,:]=np.nan
pnl_theta_open=price_open-price_lag
pnl_spot=price_spot_move-price_open
pnl_vol=price-price_spot_move
pnl_spot_dn=pnl_spot-option_greeks_open['delta']*(spot_grid-spot_lag_grid)

In [187]:
pnl_spot_dn

array([[           nan,            nan,            nan, ...,
                   nan,            nan,            nan],
       [1.83821952e-02, 2.22270690e-02, 2.22270690e-02, ...,
        6.54885585e-04, 6.54885479e-04, 2.79027372e-04],
       [1.99654638e-03, 2.41614814e-03, 2.41614814e-03, ...,
        7.90114369e-05, 7.90114280e-05, 3.65909671e-05],
       ...,
       [           nan,            nan,            nan, ...,
        5.82477145e-01, 5.82477147e-01, 6.46551235e-01],
       [           nan,            nan,            nan, ...,
        4.74705730e-02, 4.74705726e-02, 5.20908502e-02],
       [           nan,            nan,            nan, ...,
        1.03107334e-02, 1.03107335e-02, 1.13739872e-02]])

In [174]:
#price=py_vollib_vectorized.models.vectorized_black_scholes_merton(pc_grid.flatten(),spot_grid.flatten(),strikes_grid.flatten(),texp_grid.flatten(),0,spot_vol_grid.flatten(),0)


In [66]:
#try interpolation
a=np.arange(1,10)
b=np.arange(1,5)

ag,bg=np.meshgrid(a,b,indexing='ij')
c=ag*ag+2*bg
c
d=interpolate.RegularGridInterpolator((a,b),c, method='linear')
d((ag,bg))


array([[ 3.,  5.,  7.,  9.],
       [ 6.,  8., 10., 12.],
       [11., 13., 15., 17.],
       [18., 20., 22., 24.],
       [27., 29., 31., 33.],
       [38., 40., 42., 44.],
       [51., 53., 55., 57.],
       [66., 68., 70., 72.],
       [83., 85., 87., 89.]])

In [98]:
a=[(x, y) for x = 1:3, y = 1:2]

SyntaxError: invalid syntax (2203935113.py, line 1)